In [ ]:
import io

import requests
import torch
from PIL import Image
from transformers import AutoModelForCausalLM, AutoProcessor, GenerationConfig

# step 1: Setup constant
device = "cuda"
dtype = torch.float16

# step 2: Load Processor and Model
processor = AutoProcessor.from_pretrained("StanfordAIMI/CheXagent-8b", trust_remote_code=True, output_attentions=True,
        output_scores=True,
        return_dict_in_generate=True)
generation_config = GenerationConfig.from_pretrained("StanfordAIMI/CheXagent-8b")
model = AutoModelForCausalLM.from_pretrained("StanfordAIMI/CheXagent-8b", torch_dtype=dtype, trust_remote_code=True).to(device=device, dtype=dtype)

/gpfs/users/restrepoda/.conda/envs/base_ml/lib/python3.12/site-packages/transformers/utils/hub.py:105: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
A new version of the following files was downloaded from https://huggingface.co/StanfordAIMI/CheXagent-8b:
- processing_chexagent.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/StanfordAIMI/CheXagent-8b:
- configuration_chexagent.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/StanfordAIMI/CheXagent-8b:
- modeling_chexagent.py
. Make sure to double-check they do not contain any added 

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
model.config

In [10]:
generation_config

GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2,
  "max_length": 512,
  "num_beams": 5
}

In [24]:
# step 3: Fetch the images
image_path = "https://upload.wikimedia.org/wikipedia/commons/3/3b/Pleural_effusion-Metastatic_breast_carcinoma_Case_166_%285477628658%29.jpg"
#images = [Image.open(io.BytesIO(requests.get(image_path).content)).convert("RGB")]
images = [Image.open('/gpfs/workdir/restrepoda/datasets/MIMIC/mimic/preproc_224x224/s53890711_7d9bf1c6-fd83ac96-aff4a21e-bef0f1a6-c35f5c60.jpg').convert("RGB")]

# step 4: Generate the Findings section
prompt = f'Describe the image'
inputs = processor(images=images, text=f" USER: <s>{prompt} ASSISTANT: <s>", return_tensors="pt").to(device=device, dtype=dtype)
output = model.generate(**inputs,
                bos_token_id=1,
                eos_token_id=2,
                max_length=512,
                num_beams=5,

                max_new_tokens=512,
                do_sample=False,
                temperature=0.01,
                output_attentions=True,
                output_scores=True,
                return_dict_in_generate=True)

response = processor.tokenizer.decode(output.sequences[0], skip_special_tokens=True)

response

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=512) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


'A right-sided pigtail catheter has been placed. There is no evidence of pneumothorax. The lungs are clear. The cardiomediastinal silhouette is within normal limits. No acute osseous abnormalities.'

In [13]:
output.keys()

odict_keys(['sequences', 'sequences_scores', 'scores', 'beam_indices', 'attentions', 'past_key_values'])